Import and create the sparksession

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, max as spark_max
from pyspark.sql.types import DoubleType
spark = SparkSession.builder.appName("Domestic Flight Delay Record Analysis").getOrCreate()

Loading the csv file from Dataset folder

In [ ]:
file_path = "./datasets/flight_dataset.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- FL_DATE: string (nullable = true)
 |-- DEP_DELAY: integer (nullable = true)
 |-- ARR_DELAY: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- ARR_TIME: double (nullable = true)

+--------+---------+---------+--------+--------+---------+---------+
| FL_DATE|DEP_DELAY|ARR_DELAY|AIR_TIME|DISTANCE| DEP_TIME| ARR_TIME|
+--------+---------+---------+--------+--------+---------+---------+
|1/1/2006|        5|       19|     350|    2475| 9.083333|12.483334|
|1/2/2006|      167|      216|     343|    2475|11.783334|15.766666|
|1/3/2006|       -7|       -2|     344|    2475| 8.883333|12.133333|
|1/4/2006|       -5|      -13|     331|    2475| 8.916667|    11.95|
|1/5/2006|       -3|      -17|     321|    2475|     8.95|11.883333|
+--------+---------+---------+--------+--------+---------+---------+
only showing top 5 rows


#### Task 1
Flights That Arrived Earlier Than Expected

In [7]:
def count_early_arrivals(df):
  return df.filter(col("ARR_DELAY") < 0).count()

print(count_early_arrivals(df))

534655


#### Task 2
Typical Departure Time for Flights Over 2000 Miles

In [6]:
def typical_departure_long_flights(df):
  return df.filter(col("DISTANCE") > 2000) \
            .groupBy("DEP_TIME") \
            .agg(count("*").alias("count")) \
            .orderBy(col("count").desc()) \
            .limit(1)
typical_departure_long_flights(df).show()

+--------+-----+
|DEP_TIME|count|
+--------+-----+
|    6.95|  164|
+--------+-----+



#### Task 3
Proportion of Flights with Arrival Delays > 60 Minutes

In [8]:

def proportion_long_arrival_delays(df):
  df_valid = df.filter(col("ARR_DELAY").isNotNull())
  total_count = df_valid.count()
  delayed_count = df_valid.filter(col("ARR_DELAY") > 60).count()
  proportion = (delayed_count / total_count) if total_count > 0 else 0
  return proportion

print(proportion_long_arrival_delays(df))


0.053066


#### Task 4
Average Airtime Before 9:00 AM

In [9]:
def avg_airtime_before_9am(df):
    return df.filter(
        (col("DEP_TIME") < 9.0) &
        col("DEP_TIME").isNotNull() &
        col("AIR_TIME").isNotNull()
    ).agg(avg("AIR_TIME").alias("avg_airtime")).collect()[0]["avg_airtime"]

print(f"{avg_airtime_before_9am(df)} minutes")

111.36120276990287 minutes


#### Task 5
Max Arrival Delay With No Departure Delay

In [12]:


def max_arrival_delay_no_dep_delay(df):
    return df.filter(
        (col("DEP_DELAY") == 0) &
        col("ARR_DELAY").isNotNull()
    ).agg(spark_max("ARR_DELAY").alias("max_arrival_delay")).collect()[0]["max_arrival_delay"]

print(f"{max_arrival_delay_no_dep_delay(df)} minutes")

232 minutes
